In [16]:
!pip install --quiet ipyleaflet rdflib requests geodaisy pyproj shapely geopandas geomet tqdm

# Overlay features to understand spatial relationships

This notebook demonstrates how you use Loc-I to pull in 2 spatial features and understand the spatial relationships.

In [30]:
import urllib, json, requests

In [31]:
# input 1: asgs sa1
input1_uri = 'http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/80106106402'
# input 2: geofabric contracted catchment
input2_uri = 'http://linked.data.gov.au/dataset/geofabric/contractedcatchment/12105364'

Fetch the locations and show them on the map

In [35]:
#you could use rdflib to do some more digging into the semantic description to get the geometry... 
import rdflib
g1=rdflib.Graph()
g1.load(input1_uri)

#for s,p,o in g:
#    print(s, p, o)

geom_uri1 = None
for row in g1.query(
            '''
            PREFIX geo: <http://www.opengis.net/ont/geosparql#>
            select ?g where { ?feature geo:hasGeometry ?g .}
            '''):
        geom_uri1 = (row.g)
#if(geom_uri1 != None):
#    print(geom_uri1)

In [36]:
#you could use rdflib to do some more digging into the semantic description to get the geometry... 
import rdflib
g2=rdflib.Graph()
g2.load(input2_uri)

#for s,p,o in g:
#    print(s, p, o)

geom_uri2 = None
for row in g2.query(
            '''
            PREFIX geo: <http://www.opengis.net/ont/geosparql#>
            select ?g where { ?feature geo:hasGeometry ?g .}
            '''):
        geom_uri2 = (row.g)
#if(geom_uri2 != None):
#    print(geom_uri2)

In [37]:
# get the geojson representation of the geom
header={'Accept': 'application/json'}
r = requests.get(geom_uri1, headers=header)
geojson_data = r.json()

params={'_view': 'centroid'}
r = requests.get(geom_uri1, headers=header, params=params)
centroid_geojson_data = r.json()

In [38]:
#draw the geometry for the feature on a map
import ipyleaflet as ipy 
import ipywidgets as ipyw
from ipyleaflet import GeoJSON, Map, Marker

x_coord = centroid_geojson_data['coordinates'][1]
y_coord = centroid_geojson_data['coordinates'][0]

map = ipy.Map(center=[x_coord, y_coord], zoom=15)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

geo_json1 = GeoJSON(data=geojson_data, style = {'color': 'red', 'opacity':1, 'weight':1.9, 'fillOpacity':0.5})
map.add_layer(geo_json1)
map

Map(center=[-35.32380086, 149.138779385], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_…

In [39]:
# get the geojson representation of the geom
header={'Accept': 'application/json'}
r = requests.get(geom_uri2, headers=header)
geojson_data2 = r.json()

params={'_view': 'centroid'}
r = requests.get(geom_uri2, headers=header, params=params)
centroid_geojson_data2 = r.json()

In [40]:
x_coord2 = centroid_geojson_data2['coordinates'][1]
y_coord2 = centroid_geojson_data2['coordinates'][0]

map2 = ipy.Map(center=[x_coord2, y_coord2], zoom=11)

geo_json2 = GeoJSON(data=geojson_data2, style = {'color': 'blue', 'opacity':1, 'weight':1.9, 'fillOpacity':0.5})
map2.add_layer(geo_json2)
map2.add_layer(geo_json1)


Show the Contracted Catchment on the map in context with the SA1 location.

In [41]:
map2

Map(center=[-35.38234692, 149.166353306], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_…

## Find the related G-NAF addresses via the Loc-I Integration API

We want to traverse the Loc-I linkset to get all G-NAF addresses contained within the SA1 location (http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/80106106402)

In [26]:
#we give the feature_type URI to the Loc-I API
feature_type = 'http://linked.data.gov.au/def/gnaf#Address'

payload = {
    'uri': input1_uri, 
    'areas': 'false',
    'proportion': 'false',
    'contains': 'false',
    'within':'true',
    'crosswalk':'true',
    'output_type' : feature_type,
    'count': 100,
    'offset': 0
}

url = "https://api.loci.cat/api/v1/location/overlaps"
r = requests.get(url, params=payload)
res = r.json()
res

{'meta': {'count': 258, 'offset': 0},
 'overlaps': [{'uri': 'http://linked.data.gov.au/dataset/gnaf-2016-05/address/GAACT714892416'},
  {'uri': 'http://linked.data.gov.au/dataset/gnaf-2016-05/address/GAACT714892417'},
  {'uri': 'http://linked.data.gov.au/dataset/gnaf-2016-05/address/GAACT714892418'},
  {'uri': 'http://linked.data.gov.au/dataset/gnaf-2016-05/address/GAACT714892419'},
  {'uri': 'http://linked.data.gov.au/dataset/gnaf-2016-05/address/GAACT714892420'},
  {'uri': 'http://linked.data.gov.au/dataset/gnaf-2016-05/address/GAACT714892421'},
  {'uri': 'http://linked.data.gov.au/dataset/gnaf-2016-05/address/GAACT714892422'},
  {'uri': 'http://linked.data.gov.au/dataset/gnaf-2016-05/address/GAACT714892423'},
  {'uri': 'http://linked.data.gov.au/dataset/gnaf-2016-05/address/GAACT714892538'},
  {'uri': 'http://linked.data.gov.au/dataset/gnaf-2016-05/address/GAACT714892539'},
  {'uri': 'http://linked.data.gov.au/dataset/gnaf-2016-05/address/GAACT714892540'},
  {'uri': 'http://linked.d

Let's go ahead and process each address to get the spatial object and add them to the map

In [27]:
#load the resources into a graph object
address_graph = rdflib.Graph()
from tqdm import tqdm

for r in tqdm(res['overlaps']):
    address_graph.load(r['uri'])

#for s,p,o in address_graph:
#    print(s, p, o)

100%|██████████| 258/258 [00:36<00:00,  7.00it/s]


In [28]:
list_of_geom_obj = []
for row in address_graph.query(
            '''
            PREFIX geo: <http://www.opengis.net/ont/geosparql#>
            select distinct ?wkt ?type ?label ?note
            where { 
                   ?f rdfs:comment ?note .
                   ?f geo:hasGeometry ?g .
                   ?g geo:asWKT ?wkt .
                   ?g dct:type ?type .
                   ?g rdfs:label ?label .
               }
            ORDER BY ?note
            '''):
        geom = (str(row.wkt), str(row.type), row.label.value, row.note.value)
        list_of_geom_obj.append(geom)
        #print(geom)

In [29]:
#setup a new leaflet map with the meshblock
map3 = ipy.Map(center=[x_coord, y_coord], zoom=16)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))
map3.add_layer(geo_json1)

import re
from pyproj import Transformer
from shapely.wkt import dumps, loads
from geomet import wkt
from shapely.ops import transform

#add layer for each wkt
for geom_obj in list_of_geom_obj:
    #print(geom_obj[0])
    m = re.search('<(.*)> (.*)', geom_obj[0])
    crs = m.group(1)
    wktdata = m.group(2)
    #transform crs to epsg 4326 if needed...
    if crs == 'http://www.opengis.net/def/crs/EPSG/0/4283':
        project = Transformer.from_crs("EPSG:4283", "EPSG:4326")            
        wktdata = loads(wktdata)
        g2 = transform(project.transform, wktdata)  # apply projection   
        wktdata = dumps(g2)
        #print(wktdata)
    geojson_data = wkt.loads(wktdata)
    #print(geojson_data)
            
    geojson = GeoJSON(data=geojson_data, style = {'color': 'blue', 'fillcolor': 'blue', 'opacity':1, 'weight':1.9, 'fillOpacity':0.1})
    map3.add_layer(geojson)
map3

Map(center=[-35.32380086, 149.138779385], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_…

Ta-da!

You can see GNAF addresses as blue markers within the feature (shown in red). 

We hope you enjoyed that little demonstration of how to traverse from a ASGS MeshBlock across to the GNAF Address based on linkset relationships registered in Loc-I.